### Exercici 1
#### Estandaritza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

A server log is a log file (or several files) automatically created and maintained by a server consisting of a list of activities it performed.

There are generally two kinds of logs: access logs and error logs. 
This example is about an access log.

Each record in the log file represents a single HTTP request.

Example extracted from the log file:

> www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:10:31 +0100] "GET /hoteles-baratos/ofertas-hotel-Club-&-Hotel-Letoonia--en-Fethiye-8460b-destinos.html HTTP/1.1" 404 3100 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; http://www.google.com/bot.html)" VLOG=-

+ Field 1 (www.akumenius.com): Host name.
+ Field 2 (66.249.76.216): IP address of the client (remote host) which made the request to the server
+ Field 3 (-): Identity of the client (the "hyphen" indicates that the requested piece of information is not available)
+ Field 4 (-): Userid of the person requesting the document as determined by HTTP authentication (if the document is not password protected, this part will be "-" just like the previous one)
+ Field 5 ([23/Feb/2014:03:10:31 +0100]): The time that the request was received 
    + Date format: [day/month/year:hour:minute:second zone] where day = 2*digit; month = 3*letter; year = 4*digit; hour = 2*digit; minute = 2*digit; second = 2*digit; zone = (`+' | `-') 4*digit
+ Field 6 ("GET /hoteles-baratos/ofertas-hotel-Club-&-Hotel-Letoonia--en-Fethiye-8460b-destinos.html #HTTP/1.1"): Request line from the client (in double quotes). It contains a great deal of useful information: 
    + method used by the client (GET)
    + resource requested (ofertas-hotel-Club-&-Hotel-Letoonia--en-Fethiye-8460b-destinos.html)  
    + protocol (HTTP/1.1) 

+ Field 7 (404): Status code that the server sends back to the client (the full list of possible status codes can be found in the HTTP specification
    + 2: successful response 
    + 3: redirection 
    + 4: error caused by the client 
    + 5: error in the server 
+ Field 8 (3100): size of the object returned to the client
+ Field 9 ("-"): Site that the client reports having been referred from (if is not referred, it will be "-")
+ Field 10 ("Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"): Identifying information that the client browser reports about itself
+ Field 11 (VLOG=-): ???

### Exercici 2
#### Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

In [1]:
import numpy as np #library for scientific computing
import pandas as pd #library for dataframes 
import re #library for regular expressions (regex)
import warnings #library to manage warnings

warnings.filterwarnings('ignore')

In [2]:
f1_Host = r'^(?P<Host>.*?)'
f2_IP = r'(?P<IP>.*?)'
f3_Identity = r'(?P<Identity>\S+)'
f4_User = r'(?P<User>\S+)'
f5_Time = r'(?P<Time>\[.*?\])'
f6_Request = r'\"(?P<Request>.*?)\"'
f7_Status = r'(?P<Status>\d{3})'
f8_Size = r'(?P<Size>\S+)'
f9_Reference = r'\"(?P<Reference>\S+)\"'
f10_Browser = r'\"(?P<Browser>.*?)\"'
f11_Unknown = r'(?P<Unknown>.*)'
Space = r'\s'
#each field has its regex expression
#P<name>: the substring matched by the group is accessible via the group name

regex = (f1_Host+Space+f2_IP+Space+f3_Identity+Space+f4_User+Space
         +f5_Time+Space+f6_Request+Space+f7_Status+Space+f8_Size+Space
         +f9_Reference+Space+f10_Browser+Space+f11_Unknown)
#complete regex expression to capture info from the access log

def parser(log_line):
    match = re.search(regex,log_line)
    return ((match.group('Host'), match.group('IP'),
             match.group('Identity'), match.group('User'),
           match.group('Time'), match.group('Request'),
           match.group('Status'), match.group('Size'),
           match.group('Reference'), match.group('Browser'),
           match.group('Unknown')
            ))
#we create a parser method to check access log capture

logLine = """localhost 127.0.0.1 - - [23/Feb/2014:03:10:31 +0100] "OPTIONS * HTTP/1.0" 200 - "-" "Apache (internal dummy connection)" VLOG=-"""
#this is an example of a record in the access log

result = parser(logLine)
print(result)
#we check that the regex expression properly returns the fields we need from the access log

('localhost', '127.0.0.1', '-', '-', '[23/Feb/2014:03:10:31 +0100]', 'OPTIONS * HTTP/1.0', '200', '-', '-', 'Apache (internal dummy connection)', 'VLOG=-')


In [3]:
df_log = pd.read_csv('Web_access_log-akumenius.com.txt',
    sep=regex,
    na_values='-',
    
    names=['f1_Host', 'f2_IP', 'f3_Identity', 'f4_User', 
           'f5_Time', 'f6_Request', 'f7_Status', 'f8_Size',
           'f9_Reference','f10_Browser','f11_Unknown','f12_NaN'])
#we create a dataframe from the data captured in the access log using the regex we made before
#the "-" values are changed to NaN


In [4]:
df_log.index = range(1, 261874, 1)
#we add an index

In [5]:
df_log.info()
#dataframe info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261873 entries, 1 to 261873
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   f1_Host       261872 non-null  object 
 1   f2_IP         261872 non-null  object 
 2   f3_Identity   0 non-null       float64
 3   f4_User       27 non-null      object 
 4   f5_Time       261872 non-null  object 
 5   f6_Request    261835 non-null  object 
 6   f7_Status     261872 non-null  float64
 7   f8_Size       219537 non-null  float64
 8   f9_Reference  162325 non-null  object 
 9   f10_Browser   261653 non-null  object 
 10  f11_Unknown   261872 non-null  object 
 11  f12_NaN       0 non-null       float64
dtypes: float64(4), object(8)
memory usage: 24.0+ MB


In [6]:
df_log.head()
#first logs from the dataframe

,f1_Host,f2_IP,f3_Identity,f4_User,f5_Time,f6_Request,f7_Status,f8_Size,f9_Reference,f10_Browser,f11_Unknown,f12_NaN
1,localhost,127.0.0.1,NaN,NaN,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200.0,NaN,NaN,Apache (internal dummy connection),VLOG=-,NaN
2,localhost,127.0.0.1,NaN,NaN,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200.0,NaN,NaN,Apache (internal dummy connection),VLOG=-,NaN
3,localhost,127.0.0.1,NaN,NaN,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200.0,NaN,NaN,Apache (internal dummy connection),VLOG=-,NaN
4,localhost,127.0.0.1,NaN,NaN,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200.0,NaN,NaN,Apache (internal dummy connection),VLOG=-,NaN
5,localhost,127.0.0.1,NaN,NaN,[23/Feb/2014:03:10:31 +0100],OPTIONS * HTTP/1.0,200.0,NaN,NaN,Apache (internal dummy connection),VLOG=-,NaN


In [7]:
df_log.isna().sum() 
#NaN amount

f1_Host              1
f2_IP                1
f3_Identity     261873
f4_User         261846
f5_Time              1
f6_Request          38
f7_Status            1
f8_Size          42336
f9_Reference     99548
f10_Browser        220
f11_Unknown          1
f12_NaN         261873
dtype: int64

In [8]:
import re

#fields 5, 6 and 10 can be split to obtain some valuable information

df_log['f5_Time_date'] = df_log['f5_Time'].str.slice(1,11)
df_log['f5_Time_hour'] = df_log['f5_Time'].str.slice(13,21)
df_log['f5_Time_zone'] = df_log['f5_Time'].str.slice(22,27)
#we create a column for date, hour and time zone using the slice method
#we can use the slice method instead regex because the time field always have the same size

df_log['f6_Request_method'] = df_log['f6_Request'].str.extract(r'(^.*?\s)')
df_log['f6_Request_URL'] = df_log['f6_Request'].str.extract(r'(\s.*?\s)')
df_log['f6_Request_protocol'] = df_log['f6_Request'].str.extract(r'([\S]+$)')
#we create a column for method, URL and protocol using the regex expression

df_log1 = df_log['f10_Browser'].str.extract(r'(^.*?\()')
df_log['f10_Browser_version'] = df_log1[0].str.strip('(')
#we create a column for the browser using the regex expression and deleting the first parenthesis

df_log.columns
#we have now these columns

Index(['f1_Host', 'f2_IP', 'f3_Identity', 'f4_User', 'f5_Time', 'f6_Request',
       'f7_Status', 'f8_Size', 'f9_Reference', 'f10_Browser', 'f11_Unknown',
       'f12_NaN', 'f5_Time_date', 'f5_Time_hour', 'f5_Time_zone',
       'f6_Request_method', 'f6_Request_URL', 'f6_Request_protocol',
       'f10_Browser_version'],
      dtype='object')

### Exercici 3
#### Geolocalitza les IP's.

In [9]:
df_log["f2_IP"].value_counts()
#it shows how many IPs have accessed the server and how many times
#we can see there are 2921 differents IPs

66.249.76.216     46382
80.28.221.123     14725
127.0.0.1         13892
217.125.71.222     5201
66.249.75.148      3558
                  ...  
46.236.23.222         1
189.130.53.227        1
217.251.135.62        1
46.236.24.53          1
202.46.61.26          1
Name: f2_IP, Length: 2921, dtype: int64

In [3]:
import json #library to work with javascript language expressions
import urllib.request #library to use internet resources

print ("Enter the IP to geolocate:")
IP_user = input()
#the user enters an IP

url = 'http://ipinfo.io/'+IP_user+'/json'
response = urllib.request.urlopen(url)
data = json.load(response)
#the users IP its checked in a specific website and returns IP geolocation

IP=data['ip']
org=data['org']
city = data['city']
country=data['country']
region=data['region']

print("IP geolocation details for the ", IP_user,"are:")
print("City: ", city)
print("Region: ", region)
print("Country: ", country)
print("Provider: ", org)

#the 10 most IP visiting the web are from: 
#66.249.76.216      46382 BE
#80.28.221.123      14725 ES
#127.0.0.1          13892 local host
#217.125.71.222      5201 ES
#66.249.75.148       3558 US
#162.243.192.191     2927 US
#62.117.197.230      2567 ES
#89.128.176.162      1093 ES
#198.143.133.154     1045 US
#176.31.255.177      1044 FR

#the IP that visits the most is from BELGIUM, the remain IPs are much less

Enter the IP to geolocate:
80.28.221.123
IP geolocation details for the  80.28.221.123 are:
City:  Madrid
Region:  Madrid
Country:  ES
Provider:  AS3352 TELEFONICA DE ESPANA


### Exercici 4
#### Mostra'm la teva creativitat, Sorprèn-me fes un pas més enllà amb l'anàlisi anterior.

In [11]:
df_log["f10_Browser_version"].value_counts().nlargest(10)
#the most popular browsers is Mozilla in different versions

Mozilla/5.0                 222005
Mozilla/4.0                  18633
Apache                       13892
facebookexternalhit/1.1       2032
Mozilla/5.0+                  1646
Wget/1.12                     1066
ia_archiver                    384
DoCoMo/2.0 N905i               329
Opera/9.80                     227
msnbot/2.0b                    161
Name: f10_Browser_version, dtype: int64

In [12]:
df_log["f6_Request_URL"].value_counts().nlargest(10)
#the most popular URL requested for this web

 *                                                                  13892
 /destinos-get                                                       8115
 /                                                                   4058
 /hotel-list-data/                                                   2342
 /modules/raton/views/themes/bcoos/images/boto_home_reserva.png      1754
 /includes/images/uploaded/logo.png                                  1740
 /modules/raton/views/themes/bcoos/images/boto_home_planea.png       1738
 /modules/raton/views/themes/bcoos/images/boto_home_elige.png        1731
 /raton-search                                                       1694
 /hotel-list                                                         1666
Name: f6_Request_URL, dtype: int64

In [13]:
df_log["f9_Reference"].value_counts().nlargest(10)
#references are from the akumenius site, there are not from other websites, which is strange

http://www.akumenius.com/                                                                                        41394
http://www.akumenius.com/hotel-list                                                                              22039
http://test.akumenius.com/newdesign/                                                                             11839
http://www.akumenius.com/hoteles-baratos/hoteles-todo-incluido.html                                               5398
http://www.akumenius.com/chollos                                                                                  4390
http://www.akumenius.com/escapadas                                                                                3708
http://www.akumenius.com/escapadas/escapada-golf-islantilla-golf-resort-0437afab49458d7bd5a48954a459ad6d.html     3057
http://www.akumenius.com/escapadas/escapada-golf-hotel-elba-estepona-0cc275f95c9f11febf96eedab9465da3.html        2164
http://www.akumenius.com/escapadas/golf-500.html

In [14]:
df_log["f7_Status"].value_counts().nlargest(10)
#we can see the status code for the server. The are a lot of redirections and error responses

200.0    226381
304.0     25269
404.0      8630
301.0       870
206.0       304
403.0       194
302.0       109
502.0        44
408.0        37
400.0        26
Name: f7_Status, dtype: int64

In [24]:
df_log_404 = df_log[df_log["f7_Status"] == 404.0]
df_log_404["f6_Request"].value_counts()
#for example for error 404 we can filter what are the webpages with more errors and prevent it
#it looks like the most common 404 error is when the user clicks on the favicon icon


GET /favicon.ico HTTP/1.1                                                                                              138
GET /hoteles-baratos/ofertas-hotel-Castillo-del-Buen-Amor-en-Topas-25543b-destinos.html HTTP/1.1                         6
GET /hoteles-baratos/ofertas-hotel-Solvasa-Valencia-en-Valencia-43240b-destinos.html HTTP/1.1                            4
GET /hoteles-baratos/ofertas-hotel-Hospederia-del-Zenete-en-La%20Calahorra-124271b-destinos.html HTTP/1.1                4
GET /hoteles-baratos/ofertas-hoteles-en-paris.html HTTP/1.1                                                              3
                                                                                                                      ... 
GET /destinos-baratos/hoteles-baratos/ofertas-hotel-Le-Suffren-en-Port%20Grimaud-198062b-destinos.html HTTP/1.1          1
GET /hoteles-baratos/ofertas-hotel-Aptoasis-Papagayo-en-CORRALEJO%20(FUERTEVENTURA)-2984912t-destinos.html HTTP/1.1      1
GET /destinos-ba

In [23]:
df_log_404["f10_Browser_version"].value_counts()
#we can also filter if these errors are associated to a specific browser

Mozilla/5.0                 8553
ia_archiver                   52
facebookexternalhit/1.1       13
Mozilla/4.0                    8
AdsBot-Google                  1
msnbot-media/1.1               1
Opera/9.80                     1
Name: f10_Browser_version, dtype: int64